<a href="https://colab.research.google.com/github/isabelklint/scrapers/blob/main/Copy_of_Text_scraper_ethiopian_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# scraper by biswajit

EXCEL_SHEET_LINK='./Ethiopian News.xlsx'
NEWS_DOMAINS=['mereja.com','bbc.com','dw.com','www.ethiopianreporter.com','twitter.com']

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from os import listdir
from os.path import isfile, join
import io
#from PyPDF2 import PdfReader

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def mereja_format(resp_content,article_text):
    soup = bs(resp_content, 'html.parser')
    articles = soup.find("div", attrs={"class","post-entry"}).findAll("p")
    for element in articles:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

def bbc_format(resp_content,article_text):
    soup = bs(resp_content, 'html.parser')
    articles = soup.find("div", attrs={"class","e2un9en1"}).find("main").findAll("div")
    for element in articles:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

def dw_format(resp_content,article_text):
    soup = bs(resp_content, 'html.parser')
    intro_text= soup.find("p", attrs={"class","intro"})
    article_text += '\n' +intro_text.get_text()
    articles = soup.find("div", attrs={"class","longText"}).findAll("p")
    for element in articles:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

#unable to access
def ethiopianreporter_format(resp_content,article_text):
    soup = bs(resp_content, 'html.parser')
    #articles = soup.find("div", attrs={"class"," td-post-content "})
    # for element in articles:
    #     article_text += '\n' + ''.join(element.findAll(text = True))
    #return article_text
    print(soup)

def read_excel_sheet():
    df= pd.read_excel(EXCEL_SHEET_LINK)
    df_limit=df.shape[0]
    for i in range(0,df_limit):
        
        url=df.iloc[i]['URL']        
        pdf_url=df.iloc[i]['Pdf Url']

        if pd.isna(url) is False or pd.isna(pdf_url) is False:

            urls_list=[]
            if pd.isna(url) is False:
                splited_url=url.split(",")
                urls_list.extend(splited_url)
                if pd.isna(pdf_url) is False:
                    urls_list.append(pdf_url)
            else:
                if pd.isna(pdf_url) is False:
                    urls_list.append(pdf_url)  
            
            urls_list_length=len(urls_list)

            str_text=''
            for j in range(0,urls_list_length):
                headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
                response = requests.get(urls_list[j],headers=headers,verify=False)

                if urls_list[j].find('mereja.com'):
                    # its mereja format
                    str_text=mereja_format(response.content,str_text)

                if urls_list[j].find('bbc.com'):
                    #its bbc format
                    str_text=bbc_format(response.content,str_text)
                if urls_list[j].find('dw.com'):
                    #its dw format
                    str_text=dw_format(response.content,str_text)
                # if urls_list[j].find('www.ethiopianreporter.com'):
                #     #its ethiopia format
                # if urls_list[j].find('twitter.com'):
                #     #twitter format    
                # if urls_list[j].find('www.amharaamerica.org/_files/') or urls_list[j].find('.pdf'):
                #     pdf file    

#read_excel_sheet()    

# headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
# response = requests.get('https://www.ethiopianreporter.com',headers=headers,verify=False)
# ethiopianreporter_format(response.content,'')